<center>
    <img src="https://rockborne.com/wp-content/uploads/2021/07/LandingPage-Header-RED-CENTRE.jpg" width="900" alt="logo"  />
</center>

---
# PySpark
---

---
# Prerequisites
- Have the following data downloaded:
  - `people.json`
  - `appl-stock.csv`
  - `Production.Product.csv`
  - `superstore.csv`

<a name="anchorWhatIsSpark" style="position:absolute;"></a>
<hr style="border:2px solid">

# Introduction
<hr style="border-top:1px dashed">

This notebook will guide you through the basics of using PySpark for data analysis.  

We will work with a dataset containing student marks across different modules.  

You'll learn how to:  
- Load and explore data using PySpark  
- Perform basic transformations such as filtering  
- Do basic data quality analysis
- JOINS, UNIONS and GROUP BY

Throughout the notebook, you’ll find **tasks** to complete on your own to reinforce your understanding.


In [191]:
# Start a SparkSession
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.5.7-bin-hadoop3'

In [192]:
# Import necessary libraries
import pyspark
import pandas as pd
from pyspark.sql import SparkSession

In [193]:
spark = SparkSession.builder.appName('DataWrangling').getOrCreate()
spark

**From a list:**

In [194]:
data_list = [('John','','Doe','1998-04-08','M',2000),
  ('Michael','Williams','','2002-05-24','M',3000),
  ('James','','Jones','1981-09-09','M',4000),
  ('Maria','Anne','Jones','1973-12-08','F',4000),
  ('Mary','Beth','Doe','1985-02-13','F',-1)]
## Define columns
columns = ["firstname","middlename","lastname","dob","gender","salary"]

data_list, columns

([('John', '', 'Doe', '1998-04-08', 'M', 2000),
  ('Michael', 'Williams', '', '2002-05-24', 'M', 3000),
  ('James', '', 'Jones', '1981-09-09', 'M', 4000),
  ('Maria', 'Anne', 'Jones', '1973-12-08', 'F', 4000),
  ('Mary', 'Beth', 'Doe', '1985-02-13', 'F', -1)],
 ['firstname', 'middlename', 'lastname', 'dob', 'gender', 'salary'])

In [195]:
df_from_list = spark.createDataFrame(data = data_list, schema = columns)
df_from_list

DataFrame[firstname: string, middlename: string, lastname: string, dob: string, gender: string, salary: bigint]

In [196]:
df_from_list.collect()

[Row(firstname='John', middlename='', lastname='Doe', dob='1998-04-08', gender='M', salary=2000),
 Row(firstname='Michael', middlename='Williams', lastname='', dob='2002-05-24', gender='M', salary=3000),
 Row(firstname='James', middlename='', lastname='Jones', dob='1981-09-09', gender='M', salary=4000),
 Row(firstname='Maria', middlename='Anne', lastname='Jones', dob='1973-12-08', gender='F', salary=4000),
 Row(firstname='Mary', middlename='Beth', lastname='Doe', dob='1985-02-13', gender='F', salary=-1)]

### **Basic Functions**

**Showing the data:**

In [197]:
df_from_list.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|     John|          |     Doe|1998-04-08|     M|  2000|
|  Michael|  Williams|        |2002-05-24|     M|  3000|
|    James|          |   Jones|1981-09-09|     M|  4000|
|    Maria|      Anne|   Jones|1973-12-08|     F|  4000|
|     Mary|      Beth|     Doe|1985-02-13|     F|    -1|
+---------+----------+--------+----------+------+------+



**Looking at data architecture:**

In [198]:
df_from_list.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)



<a name="anchorPySparkSQL" style="position:absolute;"></a>
<hr style="border:2px solid">

# PySpark SQL and Data Wrangling
<hr style="border-top:1px dashed">

**Why Define Schema?**

Defining a data's schema has two main advantages:<br>
- the user can ensure that the dtypes for each column is correct
- the load-in times for data are much faster when spark does not have to try and infer the schema for each column

In [199]:
df_from_list.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|     John|          |     Doe|1998-04-08|     M|  2000|
|  Michael|  Williams|        |2002-05-24|     M|  3000|
|    James|          |   Jones|1981-09-09|     M|  4000|
|    Maria|      Anne|   Jones|1973-12-08|     F|  4000|
|     Mary|      Beth|     Doe|1985-02-13|     F|    -1|
+---------+----------+--------+----------+------+------+



In [200]:
df_from_list

DataFrame[firstname: string, middlename: string, lastname: string, dob: string, gender: string, salary: bigint]

In [11]:
#fixing schema
#from pyspark.sql.types import StructField, StringType, IntegerType, StructType
from pyspark.sql.types import *
list_data_schema = [StructField("firstname", StringType(), True),
                    StructField("middlename", StringType(), True),
                    StructField("lastname", StringType(), True),
                    StructField("dob", StringType(), True),
                    StructField("gender", StringType(), True),
                    StructField("salary", IntegerType(), True)]
final_struc = StructType(fields=list_data_schema)
final_struc

StructType([StructField('firstname', StringType(), True), StructField('middlename', StringType(), True), StructField('lastname', StringType(), True), StructField('dob', StringType(), True), StructField('gender', StringType(), True), StructField('salary', IntegerType(), True)])

In [12]:
df = spark.createDataFrame(data = data_list, schema = final_struc)

In [13]:
df.show()

+---------+----------+--------+----------+------+------+
|firstname|middlename|lastname|       dob|gender|salary|
+---------+----------+--------+----------+------+------+
|     John|          |     Doe|1998-04-08|     M|  2000|
|  Michael|  Williams|        |2002-05-24|     M|  3000|
|    James|          |   Jones|1981-09-09|     M|  4000|
|    Maria|      Anne|   Jones|1973-12-08|     F|  4000|
|     Mary|      Beth|     Doe|1985-02-13|     F|    -1|
+---------+----------+--------+----------+------+------+



In [14]:
df

DataFrame[firstname: string, middlename: string, lastname: string, dob: string, gender: string, salary: int]

---
<a name="anchorFunctions" style="position:absolute;"></a>

## Basic Transformations
---

**Double check that the `appl_stock.csv` file is uploaded into the Files area** - upload again now if it is not (refer to the start of the notebook if you need a refresh on how to upload).

In [ ]:
# Let Spark know about the header and infer the Schema types!
#df = spark.read.csv('../DataSources/appl_stock.csv',inferSchema=True,header=True)
#reps = pd.read_csv('../DataSources/sales_reps.csv')

In [201]:
##AWS S3 Bucket:
import pandas as pd
url = "https://rockborne-bucket-01-cbs.s3.eu-west-2.amazonaws.com/DataSources/appl_stock.csv"
# Read with pandas
pandas_df = pd.read_csv(url)
# Convert to PySpark
df = spark.createDataFrame(pandas_df)
df.show()

+----------+----------+------------------+----------+----------+---------+------------------+
|      Date|      Open|              High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+------------------+----------+----------+---------+------------------+
|2010-01-04|213.429998|        214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|        215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|            215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|        212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|        212.000006|209.060005|211.980005|111902700|         27.464034|
|2010-01-11|212.799997|        213.000002|208.450005|210.110003|115557400|         27.221758|
|2010-01-12|209.189995|        209.769995|206.419998|207.720001|148614900|          26.91211|
|2010-01-13|207.870005|        210.929995|204.099998|210.650

In [202]:
df

DataFrame[Date: string, Open: double, High: double, Low: double, Close: double, Volume: bigint, Adj Close: double]

In [203]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Adj Close: double (nullable = true)



In [204]:
df.show(5)

+----------+----------+----------+----------+----------+---------+------------------+
|      Date|      Open|      High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+----------+----------+----------+---------+------------------+
|2010-01-04|213.429998|214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|    215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|212.000006|209.060005|211.980005|111902700|         27.464034|
+----------+----------+----------+----------+----------+---------+------------------+
only showing top 5 rows



In [205]:
df.count()

1762

In [206]:
df.describe().show()

+-------+----------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|summary|      Date|             Open|              High|              Low|             Close|             Volume|         Adj Close|
+-------+----------+-----------------+------------------+-----------------+------------------+-------------------+------------------+
|  count|      1762|             1762|              1762|             1762|              1762|               1762|              1762|
|   mean|      NULL|313.0763111589103|315.91128801645857|309.8282405079455| 312.9270656379114|9.422577587968218E7| 75.00174115607265|
| stddev|      NULL|185.2994680398154|186.89817686485776|183.3839166437098|185.14710361709436|6.020518776592714E7|28.574929721799045|
|    min|2010-01-04|             90.0|         90.699997|        89.470001|         90.279999|           11475900|         24.881912|
|    max|2016-12-30|       702.409988|        705.070023|     

In [207]:
from pyspark.sql import functions as F

In [208]:
#from pyspark.sql import functions as F
## Select *
#df.select("*").show()
##Option 1:
#df.select("Date", "Open", "Volume").show()
##Option 2:
#df.select(F.col("Date"), F.col("Open"), F.col("Volume")).show()
##Option 3:
#df.select(["Date", "Open", "Volume"]).show()

##Aliases: 
#df.select(F.col("Date").alias("date"), F.col("Open")).show()
#df.select(F.col("Date").alias("date here"), F.col("Open")).show()

##Distinct
#df.select("Date").distinct().count(), df.count()

# Select columns with expressions
#f.select("Volume", (F.col("Volume") * 1.1).alias("Volume + 10%")).show()


#### **Filtering Data**

Spark DataFrames are built on top of the Spark SQL platform, which means that if you already know SQL, you can quickly and easily grab that data using SQL commands. DataFrame methods can also be used if one prefers.

References:

https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.html
https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.filter.html#pyspark.sql.DataFrame.filter
https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.DataFrame.where.html#pyspark.sql.DataFrame.where

In [209]:
# Using SQL filter rows with "Close" value below 500
df.filter("Close<500").show()

+----------+----------+------------------+----------+----------+---------+------------------+
|      Date|      Open|              High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+------------------+----------+----------+---------+------------------+
|2010-01-04|213.429998|        214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|        215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|            215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|        212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|        212.000006|209.060005|211.980005|111902700|         27.464034|
|2010-01-11|212.799997|        213.000002|208.450005|210.110003|115557400|         27.221758|
|2010-01-12|209.189995|        209.769995|206.419998|207.720001|148614900|          26.91211|
|2010-01-13|207.870005|        210.929995|204.099998|210.650

<div class="alert alert-block alert-warning">
<b>Note</b>
This is the same as a SQL WHERE clause<br>
    
> <span style="color:purple;font-weight:bold">SELECT</span> *<br>
<span style="color:purple;font-weight:bold">FROM</span> df<br>
<span style="color:purple;font-weight:bold">WHERE</span> Close < 500
    </div>

In [210]:
# Using SQL filter rows with "Close" value below 500 showing just the Open column
df.filter("Close<500").select('Open').show()

+----------+
|      Open|
+----------+
|213.429998|
|214.599998|
|214.379993|
|    211.75|
|210.299994|
|212.799997|
|209.189995|
|207.870005|
|210.110003|
|210.929995|
|208.330002|
|214.910006|
|212.079994|
|206.780006|
|202.510002|
|205.950001|
|206.849995|
|204.930004|
|201.079996|
|192.369997|
+----------+
only showing top 20 rows



<div class="alert alert-block alert-warning">
<b>Note</b>
This is the same as a SQL WHERE clause<br>
    
> <span style="color:purple;font-weight:bold">SELECT</span> Open<br>
<span style="color:purple;font-weight:bold">FROM</span> df<br>
<span style="color:purple;font-weight:bold">WHERE</span> Close < 500
    </div>

In [211]:
# Using SQL filter rows with "Close" value below 500 showing "Open" and "Close" columns
df.filter("Close<500").select(['Open','Close']).show(5)

+----------+----------+
|      Open|     Close|
+----------+----------+
|213.429998|214.009998|
|214.599998|214.379993|
|214.379993|210.969995|
|    211.75|    210.58|
|210.299994|211.980005|
+----------+----------+
only showing top 5 rows



<div class="alert alert-block alert-warning">
<b>Note</b>
This is the same as a SQL WHERE clause<br>
    
> <span style="color:purple;font-weight:bold">SELECT</span> Open, Close<br>
<span style="color:purple;font-weight:bold">FROM</span> df<br>
<span style="color:purple;font-weight:bold">WHERE</span> Close < 500<br>
<span style="color:purple;font-weight:bold">LIMIT</span> 5
    </div>

Using normal python comparison operators is another way to do this, they will look very similar to SQL operators, except you need to make sure you are calling the entire column within the dataframe, using the format: df["column name"]

In [212]:
##Filtering rows with "Close" less than 200 with the structure "df['column_name']"
df.filter(df["Close"] < 200).show(5)

+----------+----------+----------+----------+----------+---------+---------+
|      Date|      Open|      High|       Low|     Close|   Volume|Adj Close|
+----------+----------+----------+----------+----------+---------+---------+
|2010-01-22|206.780006|207.499996|    197.16|    197.75|220441900|25.620401|
|2010-01-28|204.930004|205.500004|198.699995|199.289995|293375600|25.819922|
|2010-01-29|201.079996|202.199995|190.250002|192.060003|311488100|24.883208|
|2010-02-01|192.369997|     196.0|191.299999|194.729998|187469100|25.229131|
|2010-02-02|195.909998|196.319994|193.379993|195.859997|174585600|25.375533|
+----------+----------+----------+----------+----------+---------+---------+
only showing top 5 rows



<div class="alert alert-block alert-warning">
<b>Note</b>
This is the same as a SQL WHERE clause<br>
    
> <span style="color:purple;font-weight:bold">SELECT</span> *<br>
<span style="color:purple;font-weight:bold">FROM</span> df<br>
<span style="color:purple;font-weight:bold">WHERE</span> Close < 500<br>
<span style="color:purple;font-weight:bold">LIMIT</span> 5
    </div>

**Using Boolean Expressions**

<div class="alert alert-block alert-info">
    <b>Note:</b> The next produce will cause an error, make sure to read the error!
</div>

In [213]:
df.filter(df["Close"] < 200 and df['Open'] > 200).show()

PySparkValueError: [CANNOT_CONVERT_COLUMN_INTO_BOOL] Cannot convert column into bool: please use '&' for 'and', '|' for 'or', '~' for 'not' when building DataFrame boolean expressions.

**What does this error tells us about the syntax?**

The right way:

In [214]:
df.filter((df["Close"] < 200) & (df['Open'] > 200)).show()

+----------+----------+----------+----------+----------+---------+---------+
|      Date|      Open|      High|       Low|     Close|   Volume|Adj Close|
+----------+----------+----------+----------+----------+---------+---------+
|2010-01-22|206.780006|207.499996|    197.16|    197.75|220441900|25.620401|
|2010-01-28|204.930004|205.500004|198.699995|199.289995|293375600|25.819922|
|2010-01-29|201.079996|202.199995|190.250002|192.060003|311488100|24.883208|
+----------+----------+----------+----------+----------+---------+---------+



In [215]:
# Make sure to add in the parenthesis separating the statements!
df.filter( (df["Close"] < 200) & (df['Open'] > 200) ).show()

+----------+----------+----------+----------+----------+---------+---------+
|      Date|      Open|      High|       Low|     Close|   Volume|Adj Close|
+----------+----------+----------+----------+----------+---------+---------+
|2010-01-22|206.780006|207.499996|    197.16|    197.75|220441900|25.620401|
|2010-01-28|204.930004|205.500004|198.699995|199.289995|293375600|25.819922|
|2010-01-29|201.079996|202.199995|190.250002|192.060003|311488100|24.883208|
+----------+----------+----------+----------+----------+---------+---------+



In [216]:
# ***Task***
# Get the same results as above using SQL context

#----------------------------------------------------------------------#

df.filter("Close < 200 AND Open > 200").show()
#----------------------------------------------------------------------#

+----------+----------+----------+----------+----------+---------+---------+
|      Date|      Open|      High|       Low|     Close|   Volume|Adj Close|
+----------+----------+----------+----------+----------+---------+---------+
|2010-01-22|206.780006|207.499996|    197.16|    197.75|220441900|25.620401|
|2010-01-28|204.930004|205.500004|198.699995|199.289995|293375600|25.819922|
|2010-01-29|201.079996|202.199995|190.250002|192.060003|311488100|24.883208|
+----------+----------+----------+----------+----------+---------+---------+



<details>
    <summary style="color:green;font-weight:bold">Click here for the answers to the task</summary>
    
>
```
df.filter("Close < 200 AND Open > 200").show()
```
</details>

In [217]:
#***Task***
# Display the rows with "Close" value less than 200 OR "Open" greater than 200
# Using python comparison operators

#------------------------------------------------------------------------------#
df.filter( (df["Close"] < 200) | (df['Open'] > 200) ).show()

#------------------------------------------------------------------------------#

+----------+----------+------------------+----------+----------+---------+------------------+
|      Date|      Open|              High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+------------------+----------+----------+---------+------------------+
|2010-01-04|213.429998|        214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|        215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|            215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|        212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|        212.000006|209.060005|211.980005|111902700|         27.464034|
|2010-01-11|212.799997|        213.000002|208.450005|210.110003|115557400|         27.221758|
|2010-01-12|209.189995|        209.769995|206.419998|207.720001|148614900|          26.91211|
|2010-01-13|207.870005|        210.929995|204.099998|210.650

<details>
    <summary style="color:green;font-weight:bold">Click here for the answers to the task</summary>
    
>
```
df.filter( (df["Close"] < 200) | (df['Open'] > 200) ).show()
```
</details>

In [218]:
# ***Task***
# Get the same results as above using SQL context

#----------------------------------------------------------------------#
df.filter("Close < 200 OR Open > 200" ).show()

#----------------------------------------------------------------------#

+----------+----------+------------------+----------+----------+---------+------------------+
|      Date|      Open|              High|       Low|     Close|   Volume|         Adj Close|
+----------+----------+------------------+----------+----------+---------+------------------+
|2010-01-04|213.429998|        214.499996|212.380001|214.009998|123432400|         27.727039|
|2010-01-05|214.599998|        215.589994|213.249994|214.379993|150476200|         27.774976|
|2010-01-06|214.379993|            215.23|210.750004|210.969995|138040000|27.333178000000004|
|2010-01-07|    211.75|        212.000006|209.050005|    210.58|119282800|          27.28265|
|2010-01-08|210.299994|        212.000006|209.060005|211.980005|111902700|         27.464034|
|2010-01-11|212.799997|        213.000002|208.450005|210.110003|115557400|         27.221758|
|2010-01-12|209.189995|        209.769995|206.419998|207.720001|148614900|          26.91211|
|2010-01-13|207.870005|        210.929995|204.099998|210.650

<details>
    <summary style="color:green;font-weight:bold">Click here for the answers to the task</summary>
    
>
```
df.filter("Close < 200 OR Open > 200" ).show()
```
</details>

In [219]:
##Rows with "Close" less than 200 and "Open" not less than 200
df.filter( (df["Close"] < 200) & ~(df['Open'] < 200) ).show()

+----------+----------+----------+----------+----------+---------+---------+
|      Date|      Open|      High|       Low|     Close|   Volume|Adj Close|
+----------+----------+----------+----------+----------+---------+---------+
|2010-01-22|206.780006|207.499996|    197.16|    197.75|220441900|25.620401|
|2010-01-28|204.930004|205.500004|198.699995|199.289995|293375600|25.819922|
|2010-01-29|201.079996|202.199995|190.250002|192.060003|311488100|24.883208|
+----------+----------+----------+----------+----------+---------+---------+



In [220]:
# ***Task***
# Get the same results as above using SQL context

#----------------------------------------------------------------------#

df.filter( "Close < 200 AND NOT Open < 200").show()
#----------------------------------------------------------------------#

+----------+----------+----------+----------+----------+---------+---------+
|      Date|      Open|      High|       Low|     Close|   Volume|Adj Close|
+----------+----------+----------+----------+----------+---------+---------+
|2010-01-22|206.780006|207.499996|    197.16|    197.75|220441900|25.620401|
|2010-01-28|204.930004|205.500004|198.699995|199.289995|293375600|25.819922|
|2010-01-29|201.079996|202.199995|190.250002|192.060003|311488100|24.883208|
+----------+----------+----------+----------+----------+---------+---------+



<details>
    <summary style="color:green;font-weight:bold">Click here for the answers to the task</summary>
    
>
```
df.filter( "Close < 200 AND NOT Open < 200").show()
```
</details>

In [221]:
#***Task***
# Rows with "Low" equal to 197.16

#-------------------------------------------------------------------#

df.filter(df["Low"] == 197.16).show()
#-------------------------------------------------------------------#

+----------+----------+----------+------+------+---------+---------+
|      Date|      Open|      High|   Low| Close|   Volume|Adj Close|
+----------+----------+----------+------+------+---------+---------+
|2010-01-22|206.780006|207.499996|197.16|197.75|220441900|25.620401|
+----------+----------+----------+------+------+---------+---------+



<details>
    <summary style="color:green;font-weight:bold">Click here for the answers to the task</summary>
    
>
```
df.filter(df["Low"] == 197.16).show()
```
</details>

In [222]:
# ***Task***
# Get the same results as above using SQL context

#----------------------------------------------------------------------#

df.filter("Low == 197.16").show()
#----------------------------------------------------------------------#

+----------+----------+----------+------+------+---------+---------+
|      Date|      Open|      High|   Low| Close|   Volume|Adj Close|
+----------+----------+----------+------+------+---------+---------+
|2010-01-22|206.780006|207.499996|197.16|197.75|220441900|25.620401|
+----------+----------+----------+------+------+---------+---------+



<details>
    <summary style="color:green;font-weight:bold">Click here for the answers to the task</summary>
    
>
```
df.filter("Low == 197.16").show()
```
</details>

Instead of just showing our outputs, we can also collect them as <u> Python objects </u>

In [223]:
# Collecting results as Python objects
# Collect  rows with "Low" equal to 197.16
df.filter(df["Low"] == 197.16).collect()

[Row(Date='2010-01-22', Open=206.780006, High=207.499996, Low=197.16, Close=197.75, Volume=220441900, Adj Close=25.620401)]

---
<a name="anchorAggregations" style="position:absolute;"></a>
## Group By and Aggregate Functions
---

**Double check that the `superstore.csv` file is uploaded into the Files area** - upload again now if it is not (refer to the start of the notebook if you need a refresh on how to upload).

In [ ]:
from pyspark.sql import functions as F
#df = spark.read.csv('../DataSources/superstore.csv',inferSchema=True,header=True)

In [224]:
from pyspark.sql import functions as F
##AWS S3 Bucket:
import pandas as pd
url = "https://rockborne-bucket-01-cbs.s3.eu-west-2.amazonaws.com/DataSources/superstore.csv"
# Read with pandas
pandas_df = pd.read_csv(url, encoding='latin-1')
# Convert to PySpark
df = spark.createDataFrame(pandas_df)
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   Sales|Quantity|Discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [225]:
df.printSchema()

root
 |-- Row ID: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)



If your schema is incorrect (e.g., sales, quantity, or discount are stored as strings), you can cast them to the correct data types using the `.withColumn()` method and the `cast()` function from `pyspark.sql.functions`.

In [226]:
#from pyspark.sql.functions import col
df = df.withColumn("sales", F.col("sales").cast("double")) \
       .withColumn("quantity", F.col("quantity").cast("int")) \
       .withColumn("discount", F.col("discount").cast("double"))

In [227]:
df.printSchema()

root
 |-- Row ID: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- sales: double (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- discount: double (nullable = true)
 |-- Profit: double (nullable = true)



In [228]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   sales|quantity|discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [229]:
# Mean
#df.groupBy("Ship Mode").mean().show()
df.groupBy("Ship Mode").mean().show()

+--------------+------------------+------------------+------------------+------------------+-------------------+------------------+
|     Ship Mode|       avg(Row ID)|  avg(Postal Code)|        avg(sales)|     avg(quantity)|      avg(discount)|       avg(Profit)|
+--------------+------------------+------------------+------------------+------------------+-------------------+------------------+
|   First Class| 4875.510403120937|54765.611833550065|228.49702399219763| 3.701560468140442| 0.1646098829648895| 31.83994791937582|
|      Same Day|5128.9097605893185| 57536.08287292818|236.39617863720073|3.6095764272559854|0.15239410681399626|29.266590976058925|
|  Second Class| 4936.759383033419| 55626.00925449871| 236.0892387660668| 3.816452442159383| 0.1388946015424165|29.535545192802065|
|Standard Class| 5036.776977211796| 54944.44705093834|227.58306685656837| 3.819872654155496|0.16002345844504015| 27.49477002345845|
+--------------+------------------+------------------+------------------+---

In [230]:
# Count
df.groupBy("Ship Mode").count().show()

+--------------+-----+
|     Ship Mode|count|
+--------------+-----+
|   First Class| 1538|
|      Same Day|  543|
|  Second Class| 1945|
|Standard Class| 5968|
+--------------+-----+



In [231]:
# Sum
df.groupBy("Ship Mode").sum().show()

+--------------+-----------+----------------+------------------+-------------+------------------+------------------+
|     Ship Mode|sum(Row ID)|sum(Postal Code)|        sum(sales)|sum(quantity)|     sum(discount)|       sum(Profit)|
+--------------+-----------+----------------+------------------+-------------+------------------+------------------+
|   First Class|    7498535|        84229511|351428.42289999995|         5693|253.17000000000002| 48969.83990000001|
|      Same Day|    2784998|        31242093|        128363.125|         1960| 82.74999999999997|15891.758899999997|
|  Second Class|    9601997|       108192588|       459193.5694|         7423| 270.1500000000001|57446.635400000014|
|Standard Class|   30059485|       327908460|       1358215.743|        22797| 955.0199999999996|164088.78750000003|
+--------------+-----------+----------------+------------------+-------------+------------------+------------------+



Max of `sales` from the whole dataframe:

Dictionary used to specify `sales` column to aggregate.

In [232]:
df.agg({'sales':'max'}).show()

+----------+
|max(sales)|
+----------+
|  22638.48|
+----------+



In [233]:
# Get the average profit for each state
df.groupBy("State").agg({"Profit": "avg"}).show()

+--------------------+-------------------+
|               State|        avg(Profit)|
+--------------------+-------------------+
|                Utah| 48.047801886792456|
|           Minnesota| 121.60884719101122|
|                Ohio| -36.18630405117271|
|              Oregon| -9.600568548387097|
|            Arkansas|  66.81145166666667|
|               Texas|-26.121173908629444|
|        Pennsylvania|-26.507598466780237|
|         Connecticut|  42.82307073170732|
|            Nebraska|  53.60774210526316|
|              Nevada|  85.04527948717949|
|          Washington|  66.01314565217392|
|            Illinois|-25.625786585365855|
|            Oklahoma|  73.54478787878787|
|District of Columbia| 105.95893000000001|
|            Delaware| 103.93098749999997|
|          New Mexico| 31.273408108108107|
|            Missouri|  97.51834090909091|
|        Rhode Island| 130.10052321428572|
|             Georgia|  88.31545271739131|
|             Montana|           122.2219|
+----------

In [234]:
#***Task***
# Get the count of orders in each region

#------------------------------------------------------------#

df.groupBy("Region").count().show()
#------------------------------------------------------------#

+-------+-----+
| Region|count|
+-------+-----+
|  South| 1620|
|Central| 2323|
|   East| 2848|
|   West| 3203|
+-------+-----+



<details>
    <summary style="color:green;font-weight:bold">Click here for the answer to the task</summary>
    
>
```
df.groupBy("Region").count().show()
```
</details>

In [235]:
#***Task***
# Find the highest order value per customer

#----------------------------------------------------------------#


df.groupBy("Customer ID").agg({"sales": "max"}).show()
#----------------------------------------------------------------#

+-----------+----------+
|Customer ID|max(sales)|
+-----------+----------+
|   VW-21775|   1247.64|
|   EM-13960|     572.8|
|   MY-17380|   801.568|
|   KH-16630|   1259.93|
|   BD-11500|   2939.93|
|   SW-20275|    881.93|
|   PH-18790|   683.952|
|   JF-15490|   291.168|
|   JF-15415|   1471.96|
|   IM-15070|   1819.86|
|   JH-15985|  4297.644|
|   OT-18730|  1369.764|
|   KM-16225|    601.65|
|   MG-18145|   1115.91|
|   SO-20335|  957.5775|
|   DN-13690|    842.72|
|   DJ-13420|    1454.9|
|   JK-15370|   573.174|
|   CB-12025|   999.432|
|   NB-18655|   383.438|
+-----------+----------+
only showing top 20 rows



<details>
    <summary style="color:green;font-weight:bold">Click here for the answer to the task</summary>
    
>
```
df.groupBy("Customer ID").agg({"sales": "max"}).show()
```
</details>

**SQL style**

In [236]:
#df.select(count_distinct("Sub-Category")).show()
df.select(F.count_distinct("Sub-Category")).show()

+----------------------------+
|count(DISTINCT Sub-Category)|
+----------------------------+
|                          17|
+----------------------------+



In [237]:
##STD of sales
sales_std = df.select(F.stddev("sales").alias('std'))

In [238]:
sales_std.show()

+-----------------+
|              std|
+-----------------+
|623.2451005086809|
+-----------------+



In [239]:
# OrderBy (Ascending)
df.orderBy("sales").show()

+------+--------------+----------+----------+--------------+-----------+----------------+-----------+-------------+-------------+------------+-----------+-------+---------------+---------------+------------+--------------------+-----+--------+--------+-------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|   Customer Name|    Segment|      Country|         City|       State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|sales|quantity|discount| Profit|
+------+--------------+----------+----------+--------------+-----------+----------------+-----------+-------------+-------------+------------+-----------+-------+---------------+---------------+------------+--------------------+-----+--------+--------+-------+
|  4102|US-2017-102288| 6/19/2017| 6/23/2017|Standard Class|   ZC-21910|Zuschuss Carroll|   Consumer|United States|      Houston|       Texas|      77095|Central|OFF-AP-10002906|Office Supplies|  Appliances|Hoover Rep

In [240]:
# Descending call off the column itself.
#df.orderBy(df["sales"].desc()).show()

# Alternatively
#df.orderBy('sales', ascending=False).show()

# Alternatively
df.orderBy(F.col('sales').desc()).show()



+------+--------------+----------+----------+--------------+-----------+--------------------+-----------+-------------+-------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+---------+--------+--------+----------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|       Customer Name|    Segment|      Country|         City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|    sales|quantity|discount|    Profit|
+------+--------------+----------+----------+--------------+-----------+--------------------+-----------+-------------+-------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+---------+--------+--------+----------+
|  2698|CA-2014-145317| 3/18/2014| 3/23/2014|Standard Class|   SM-20320|         Sean Miller|Home Office|United States| Jacksonville|       Florida|      32216|  South|TEC-MA-100

In [241]:
#***Task***
# Find the total profit per sub-category and sort the results in descending order

#-----------------------------------------------------------------------------#

df.groupBy("Sub-Category").agg({"profit": "sum"}).orderBy("sum(profit)", ascending=False).show()
#-----------------------------------------------------------------------------#

+------------+-------------------+
|Sub-Category|        sum(profit)|
+------------+-------------------+
|     Copiers|  55617.82490000001|
|      Phones| 44515.730599999995|
| Accessories|  41936.63569999999|
|       Paper|         34053.5693|
|     Binders|         30221.7633|
|      Chairs| 26590.166300000004|
|     Storage| 21278.826399999998|
|  Appliances|         18138.0054|
| Furnishings|         13059.1436|
|   Envelopes|          6964.1767|
|         Art|  6527.787000000002|
|      Labels|  5546.254000000001|
|    Machines| 3384.7568999999994|
|   Fasteners|           949.5182|
|    Supplies|-1189.0995000000003|
|   Bookcases|-3472.5559999999996|
|      Tables|-17725.481099999997|
+------------+-------------------+



<details>
    <summary style="color:green;font-weight:bold">Click here for the answer to the task</summary>
    
>
```
df.groupBy("Sub-Category").agg({"profit": "sum"}).orderBy("sum(profit)", ascending=False).show()
```
</details>

In [242]:
df.show()

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|   sales|quantity|discount|  Profit|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+--------+--------+--------+--------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|   

In [243]:
##Multiple columns
df.groupBy("Category", "Sub-Category").agg(
                                                F.count("Category").alias("Num of Orders"),
                                                F.avg("Sales").alias("avg_sales")
                                            ).orderBy(F.col("Category").desc(), F.col("avg_sales").desc()).show()


+---------------+------------+-------------+------------------+
|       Category|Sub-Category|Num of Orders|         avg_sales|
+---------------+------------+-------------+------------------+
|     Technology|     Copiers|           68|2198.9416176470586|
|     Technology|    Machines|          115|1645.5533130434783|
|     Technology|      Phones|          889| 371.2115343082115|
|     Technology| Accessories|          775| 215.9746038709677|
|Office Supplies|     Storage|          846| 264.5905531914893|
|Office Supplies|    Supplies|          190|245.65019999999996|
|Office Supplies|  Appliances|          466|230.75571030042917|
|Office Supplies|     Binders|         1523|133.56056007879187|
|Office Supplies|   Envelopes|          254| 64.86772440944883|
|Office Supplies|       Paper|         1370|57.284091970802905|
|Office Supplies|      Labels|          364| 34.30305494505494|
|Office Supplies|         Art|          796|34.068834170854274|
|Office Supplies|   Fasteners|          

In [244]:
##Multiple columns option 2
df.groupBy("Category", "Sub-Category").agg(
                                                F.count("Category").alias("Num of Orders"),
                                                F.avg("Sales").alias("avg_sales")
                                            ).orderBy(["Category", "avg_sales"], ascending=[True, False]).show()

+---------------+------------+-------------+------------------+
|       Category|Sub-Category|Num of Orders|         avg_sales|
+---------------+------------+-------------+------------------+
|      Furniture|      Tables|          319| 648.7947711598746|
|      Furniture|      Chairs|          617| 532.3324197730956|
|      Furniture|   Bookcases|          228|503.85963289473676|
|      Furniture| Furnishings|          957| 95.82566771159871|
|Office Supplies|     Storage|          846| 264.5905531914893|
|Office Supplies|    Supplies|          190|245.65019999999996|
|Office Supplies|  Appliances|          466|230.75571030042917|
|Office Supplies|     Binders|         1523|133.56056007879187|
|Office Supplies|   Envelopes|          254| 64.86772440944883|
|Office Supplies|       Paper|         1370|57.284091970802905|
|Office Supplies|      Labels|          364| 34.30305494505494|
|Office Supplies|         Art|          796|34.068834170854274|
|Office Supplies|   Fasteners|          

In [245]:
##Multiple columns renaming
result = df.groupBy("Category", "Sub-Category").agg(
    F.count("Category").alias("Num_of_Orders"),
    F.avg("Sales").alias("avg_sales")
)

result = result.withColumnRenamed("Category", "Cat") \
               .withColumnRenamed("Sub-Category", "SubCat")

result.orderBy(F.col("Cat").desc(), F.col("avg_sales").desc()).show()


+---------------+-----------+-------------+------------------+
|            Cat|     SubCat|Num_of_Orders|         avg_sales|
+---------------+-----------+-------------+------------------+
|     Technology|    Copiers|           68|2198.9416176470586|
|     Technology|   Machines|          115|1645.5533130434783|
|     Technology|     Phones|          889| 371.2115343082115|
|     Technology|Accessories|          775| 215.9746038709677|
|Office Supplies|    Storage|          846| 264.5905531914893|
|Office Supplies|   Supplies|          190|245.65019999999996|
|Office Supplies| Appliances|          466|230.75571030042917|
|Office Supplies|    Binders|         1523|133.56056007879187|
|Office Supplies|  Envelopes|          254| 64.86772440944883|
|Office Supplies|      Paper|         1370|57.284091970802905|
|Office Supplies|     Labels|          364| 34.30305494505494|
|Office Supplies|        Art|          796|34.068834170854274|
|Office Supplies|  Fasteners|          217|13.936774193

## Joins

Join Types in PySpark:

- Inner Join: Returns records with matching keys in both DataFrames
- Left (Outer) Join: Returns all records from left DF, matching records from right
- Right (Outer) Join: Returns all records from right DF, matching records from left
- Full (Outer) Join: Returns all records when there's a match in either DF
- Cross Join: Cartesian product of both DataFrames
- Left Semi Join: Returns records from left DF that have matches in right (no right columns)
- Left Anti Join: Returns records from left DF that have NO matches in right

Reference: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.join.html 

In [246]:
##Joins
# Create sample datasets
customers_data = [
    (1, "John Smith", "john@email.com", "New York"),
    (2, "Jane Doe", "jane@email.com", "Los Angeles"),
    (3, "Bob Johnson", "bob@email.com", "Chicago"),
    (4, "Alice Williams", "alice@email.com", "Houston")
]
customers_df = spark.createDataFrame(customers_data, ["customer_id", "name", "email", "city"])

orders_data = [
    (101, 1, "2023-01-15", 150.00, "Delivered"),
    (102, 2, "2023-01-16", 200.00, "Delivered"),
    (103, 1, "2023-01-17", 75.00, "Shipped"),
    (104, 3, "2023-01-18", 300.00, "Processing"),
    (105, 5, "2023-01-19", 125.00, "Delivered")  # Customer 5 doesn't exist
]
orders_df = spark.createDataFrame(orders_data, ["order_id", "customer_id", "order_date", "amount", "status"])

In [247]:
# Inner Join (default)
inner_join = orders_df.join(  ##First dataframe to join
    other = customers_df, ## Second dataframe
    on = orders_df.customer_id == customers_df.customer_id, ##keys to use
    how ="inner" ##Type of join
)
print("Inner Join:")
inner_join.show()

Inner Join:
+--------+-----------+----------+------+----------+-----------+-----------+--------------+-----------+
|order_id|customer_id|order_date|amount|    status|customer_id|       name|         email|       city|
+--------+-----------+----------+------+----------+-----------+-----------+--------------+-----------+
|     101|          1|2023-01-15| 150.0| Delivered|          1| John Smith|john@email.com|   New York|
|     103|          1|2023-01-17|  75.0|   Shipped|          1| John Smith|john@email.com|   New York|
|     102|          2|2023-01-16| 200.0| Delivered|          2|   Jane Doe|jane@email.com|Los Angeles|
|     104|          3|2023-01-18| 300.0|Processing|          3|Bob Johnson| bob@email.com|    Chicago|
+--------+-----------+----------+------+----------+-----------+-----------+--------------+-----------+



In [248]:
# Left Join
left_join = orders_df.join(
    customers_df,
    orders_df.customer_id == customers_df.customer_id,
    "left"
)

print("Left Join (all orders, even without customer):")
left_join.show()

Left Join (all orders, even without customer):
+--------+-----------+----------+------+----------+-----------+-----------+--------------+-----------+
|order_id|customer_id|order_date|amount|    status|customer_id|       name|         email|       city|
+--------+-----------+----------+------+----------+-----------+-----------+--------------+-----------+
|     101|          1|2023-01-15| 150.0| Delivered|          1| John Smith|john@email.com|   New York|
|     102|          2|2023-01-16| 200.0| Delivered|          2|   Jane Doe|jane@email.com|Los Angeles|
|     103|          1|2023-01-17|  75.0|   Shipped|          1| John Smith|john@email.com|   New York|
|     104|          3|2023-01-18| 300.0|Processing|          3|Bob Johnson| bob@email.com|    Chicago|
|     105|          5|2023-01-19| 125.0| Delivered|       NULL|       NULL|          NULL|       NULL|
+--------+-----------+----------+------+----------+-----------+-----------+--------------+-----------+



In [249]:
# Right Join
right_join = orders_df.join(
    customers_df,
    orders_df.customer_id == customers_df.customer_id,
    "right"
)

print("Right Join (all customers, even without orders):")
right_join.show()

Right Join (all customers, even without orders):
+--------+-----------+----------+------+----------+-----------+--------------+---------------+-----------+
|order_id|customer_id|order_date|amount|    status|customer_id|          name|          email|       city|
+--------+-----------+----------+------+----------+-----------+--------------+---------------+-----------+
|     103|          1|2023-01-17|  75.0|   Shipped|          1|    John Smith| john@email.com|   New York|
|     101|          1|2023-01-15| 150.0| Delivered|          1|    John Smith| john@email.com|   New York|
|     102|          2|2023-01-16| 200.0| Delivered|          2|      Jane Doe| jane@email.com|Los Angeles|
|     104|          3|2023-01-18| 300.0|Processing|          3|   Bob Johnson|  bob@email.com|    Chicago|
|    NULL|       NULL|      NULL|  NULL|      NULL|          4|Alice Williams|alice@email.com|    Houston|
+--------+-----------+----------+------+----------+-----------+--------------+---------------+-

In [250]:
# Full Outer Join
full_join = orders_df.join(
    customers_df,
    orders_df.customer_id == customers_df.customer_id,
    "full_outer"
)
print("Full Outer Join:")
full_join.show()

Full Outer Join:
+--------+-----------+----------+------+----------+-----------+--------------+---------------+-----------+
|order_id|customer_id|order_date|amount|    status|customer_id|          name|          email|       city|
+--------+-----------+----------+------+----------+-----------+--------------+---------------+-----------+
|     101|          1|2023-01-15| 150.0| Delivered|          1|    John Smith| john@email.com|   New York|
|     103|          1|2023-01-17|  75.0|   Shipped|          1|    John Smith| john@email.com|   New York|
|     102|          2|2023-01-16| 200.0| Delivered|          2|      Jane Doe| jane@email.com|Los Angeles|
|     104|          3|2023-01-18| 300.0|Processing|          3|   Bob Johnson|  bob@email.com|    Chicago|
|    NULL|       NULL|      NULL|  NULL|      NULL|          4|Alice Williams|alice@email.com|    Houston|
|     105|          5|2023-01-19| 125.0| Delivered|       NULL|          NULL|           NULL|       NULL|
+--------+----------

In [251]:
##Multiple Joins:

# Create product data
products_data = [
    (201, "Laptop", "Electronics", 999.99),
    (202, "Mouse", "Electronics", 29.99),
    (203, "Desk", "Furniture", 299.99),
    (204, "Chair", "Furniture", 199.99)
]

order_items_data = [
    (101, 201, 1),
    (101, 202, 2),
    (102, 203, 1),
    (103, 202, 1),
    (104, 201, 2),
    (104, 204, 1)
]

products_df = spark.createDataFrame(products_data,
    ["product_id", "product_name", "category", "price"])
    
order_items_df = spark.createDataFrame(order_items_data,
    ["order_id", "product_id", "quantity"])

# Chain multiple joins
complete_orders = orders_df \
    .join(customers_df, "customer_id", "inner") \
    .join(order_items_df, "order_id", "inner") \
    .join(products_df, "product_id", "inner") \
    .select(
        "order_id",
        "order_date",
        "product_id",
        "name",
        "email",
        "product_name",
        "category",
        "quantity",
        "price"
    )

print("Complete Order Details:")
complete_orders.show()

Complete Order Details:
+--------+----------+----------+-----------+--------------+------------+-----------+--------+------+
|order_id|order_date|product_id|       name|         email|product_name|   category|quantity| price|
+--------+----------+----------+-----------+--------------+------------+-----------+--------+------+
|     101|2023-01-15|       201| John Smith|john@email.com|      Laptop|Electronics|       1|999.99|
|     104|2023-01-18|       201|Bob Johnson| bob@email.com|      Laptop|Electronics|       2|999.99|
|     101|2023-01-15|       202| John Smith|john@email.com|       Mouse|Electronics|       2| 29.99|
|     103|2023-01-17|       202| John Smith|john@email.com|       Mouse|Electronics|       1| 29.99|
|     102|2023-01-16|       203|   Jane Doe|jane@email.com|        Desk|  Furniture|       1|299.99|
|     104|2023-01-18|       204|Bob Johnson| bob@email.com|       Chair|  Furniture|       1|199.99|
+--------+----------+----------+-----------+--------------+--------

In [252]:
complete_orders.printSchema()

root
 |-- order_id: long (nullable = true)
 |-- order_date: string (nullable = true)
 |-- product_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- price: double (nullable = true)



**Exercise:** Get the following output: 

In [253]:
##Exercise:
# Chain multiple joins
complete_orders = orders_df \
    .join(customers_df, "customer_id", "inner") \
    .join(order_items_df, "order_id", "inner") \
    .join(products_df, "product_id", "inner") \
    .select(
        "order_id",
        "order_date",
        "product_id",
        "name",
        "email",
        "product_name",
        "category",
        "quantity",
        "price",
        (F.col("quantity") * F.col("price")).alias("line_total")
    )

print("Complete Order Details:")
complete_orders.show()


# Aggregate after joins
#from pyspark.sql.functions import sum, count, collect_list

order_summary = complete_orders.groupBy(
    "order_id", "order_date", "name"
).agg(
    F.sum("line_total").alias("order_total"),
    F.count("product_id").alias("num_items")
)

print("Order Summary:")
order_summary.show(truncate=False)

Complete Order Details:
+--------+----------+----------+-----------+--------------+------------+-----------+--------+------+----------+
|order_id|order_date|product_id|       name|         email|product_name|   category|quantity| price|line_total|
+--------+----------+----------+-----------+--------------+------------+-----------+--------+------+----------+
|     101|2023-01-15|       201| John Smith|john@email.com|      Laptop|Electronics|       1|999.99|    999.99|
|     104|2023-01-18|       201|Bob Johnson| bob@email.com|      Laptop|Electronics|       2|999.99|   1999.98|
|     101|2023-01-15|       202| John Smith|john@email.com|       Mouse|Electronics|       2| 29.99|     59.98|
|     103|2023-01-17|       202| John Smith|john@email.com|       Mouse|Electronics|       1| 29.99|     29.99|
|     102|2023-01-16|       203|   Jane Doe|jane@email.com|        Desk|  Furniture|       1|299.99|    299.99|
|     104|2023-01-18|       204|Bob Johnson| bob@email.com|       Chair|  Furnit

## Unions

### Requirements for union() to work:

- Same number of columns
- Same column order
- Same data types in corresponding columns

Note: __Column names do not need to match, but the schema (column count and types) must be compatible.__

Reference: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.union.html 

In [254]:
# Union combines rows from multiple DataFrames with same schema
sales_q1 = spark.createDataFrame([
    (1, "2023-01-15", 1000),
    (2, "2023-02-20", 1500)
], ["sale_id", "date", "amount"])

sales_q2 = spark.createDataFrame([
    (3, "2023-04-10", 2000),
    (4, "2023-05-15", 1800)
], ["sale_id", "date", "amount"])

# Union (includes duplicates)
all_sales = sales_q1.union(sales_q2)
print("Union:")
all_sales.show()

Union:
+-------+----------+------+
|sale_id|      date|amount|
+-------+----------+------+
|      1|2023-01-15|  1000|
|      2|2023-02-20|  1500|
|      3|2023-04-10|  2000|
|      4|2023-05-15|  1800|
+-------+----------+------+



In [255]:
# Different column name,s ame schema
sales_q1 = spark.createDataFrame([
    (1, "2023-01-15", 1000),
    (2, "2023-02-20", 1500)
], ["sale_id", "date", "amount"])

sales_q2 = spark.createDataFrame([
    (3, "2023-04-10", 2000),
    (4, "2023-05-15", 1800)
], ["sale_id", "date", "amnt"]) ## change the name of a single column

# Union (includes duplicates)
all_sales = sales_q1.union(sales_q2)
print("Union:")
all_sales.show()

Union:
+-------+----------+------+
|sale_id|      date|amount|
+-------+----------+------+
|      1|2023-01-15|  1000|
|      2|2023-02-20|  1500|
|      3|2023-04-10|  2000|
|      4|2023-05-15|  1800|
+-------+----------+------+



In [256]:
# Same columns, differrent schema
sales_q1 = spark.createDataFrame([
    (1, "2023-01-15", 1000),
    (2, "2023-02-20", 1500)
], ["sale_id", "date", "amount"])
print(sales_q1.printSchema)


sales_q2 = spark.createDataFrame([
    (3, "2023-04-10", "hola"),
    (4, "2023-05-15", "hi")
], ["sale_id", "date", "amount"])
print(sales_q2.printSchema)


# Union (includes duplicates)
all_sales = sales_q1.union(sales_q2)
print("Union:")
all_sales.show()
print(sales_q2.printSchema)


<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: bigint]>
<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: string]>
Union:
+-------+----------+------+
|sale_id|      date|amount|
+-------+----------+------+
|      1|2023-01-15|  1000|
|      2|2023-02-20|  1500|
|      3|2023-04-10|  hola|
|      4|2023-05-15|    hi|
+-------+----------+------+

<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: string]>


In [257]:
# Same columns, differrent schema
sales_q1 = spark.createDataFrame([
    (1, "2023-01-15", 1000),
    (2, "2023-02-20", 1500)
], ["sale_id", "date", "amount"])
print(sales_q1.printSchema)


sales_q2 = spark.createDataFrame([
    (3, "2023-04-10", 2000.544554),
    (4, "2023-05-15", 1800.323232)
], ["sale_id", "date", "amount"])
print(sales_q2.printSchema)


# Union (includes duplicates)
all_sales = sales_q1.union(sales_q2)
print("Union:")
all_sales.show()
print(sales_q2.printSchema)

<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: bigint]>
<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: double]>
Union:
+-------+----------+-----------+
|sale_id|      date|     amount|
+-------+----------+-----------+
|      1|2023-01-15|     1000.0|
|      2|2023-02-20|     1500.0|
|      3|2023-04-10|2000.544554|
|      4|2023-05-15|1800.323232|
+-------+----------+-----------+

<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: double]>


In [258]:
# Same columns, differrent schema
sales_q1 = spark.createDataFrame([
    (1, "2023-01-15", 1000),
    (2, "2023-02-20", 1500)
], ["sale_id", "date", "amount"])
print(sales_q1.printSchema)


sales_q2 = spark.createDataFrame([
    (3, "2023-04-10", 2000.544554, 'hola'),
    (4, "2023-05-15", 1800.323232, 'adios')
], ["sale_id", "date", "amount", "col1"])
print(sales_q2.printSchema)


# Union (includes duplicates)
all_sales = sales_q1.union(sales_q2)
print("Union:")
all_sales.show()
print(sales_q2.printSchema)

<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: bigint]>
<bound method DataFrame.printSchema of DataFrame[sale_id: bigint, date: string, amount: double, col1: string]>


AnalysisException: [NUM_COLUMNS_MISMATCH] UNION can only be performed on inputs with the same number of columns, but the first input has 3 columns and the second input has 4 columns.;
'Union false, false
:- LogicalRDD [sale_id#14690L, date#14691, amount#14692L], false
+- LogicalRDD [sale_id#14696L, date#14697, amount#14698, col1#14699], false


## SQL Approach

**Familiarity**: SQL is widely known - enables analysts and data engineers to leverage existing skills without learning new syntax

**Readability**: Complex queries are often clearer in SQL, especially for joins, aggregations, and subqueries

**Team Collaboration**: Mixed teams (SQL experts + Python developers) can work together more easily

**Quick Prototyping**: Faster to write queries for exploration and ad-hoc analysis

**Flexibility**: Switch between SQL and DataFrame API as needed - use SQL for complex logic, DataFrame API for programmatic operations

**Same Performance**: Both approaches use the same Catalyst optimiser - no performance difference

**Bottom Line**: Use SQL when it's more natural/readable; use DataFrame API when you need programmatic control or dynamic operations.

**Reference:** https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.createOrReplaceTempView.html 


In [ ]:
##SQL approach
superstore_df = spark.read.csv('../DataSources/superstore.csv',inferSchema=True,header=True)
superstore_df.printSchema()
superstore_df = superstore_df.withColumn("sales", F.col("sales").cast("double")) \
       .withColumn("quantity", F.col("quantity").cast("int")) \
       .withColumn("discount", F.col("discount").cast("double"))
superstore_df.printSchema()
superstore_df.show()

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: integer (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: string (nullable = true)
 |-- Quantity: string (nullable = true)
 |-- Discount: string (nullable = true)
 |-- Profit: double (nullable = true)

root
 |-- Row ID: integer (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: str

In [ ]:
from pyspark.sql import functions as F
##AWS S3 Bucket:
import pandas as pd
url = "https://rockborne-bucket-01-cbs.s3.eu-west-2.amazonaws.com/DataSources/superstore.csv"
# Read with pandas
pandas_df = pd.read_csv(url, encoding='latin-1')
# Convert to PySpark
superstore_df = spark.createDataFrame(pandas_df)
superstore_df.printSchema()
superstore_df = superstore_df.withColumn("sales", F.col("sales").cast("double")) \
       .withColumn("quantity", F.col("quantity").cast("int")) \
       .withColumn("discount", F.col("discount").cast("double"))
superstore_df.printSchema()
superstore_df.show()

root
 |-- Row ID: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)

root
 |-- Row ID: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullab

In [ ]:
##Creating the view
superstore_df.createOrReplaceTempView("superstore")

In [ ]:
##General query:

spark.sql("SELECT * " \
"           FROM superstore " \
"           LIMIT 2").show()

+------+--------------+----------+----------+------------+-----------+-------------+--------+-------------+---------+--------+-----------+------+---------------+---------+------------+--------------------+------+--------+--------+-------+
|Row ID|      Order ID|Order Date| Ship Date|   Ship Mode|Customer ID|Customer Name| Segment|      Country|     City|   State|Postal Code|Region|     Product ID| Category|Sub-Category|        Product Name| sales|quantity|discount| Profit|
+------+--------------+----------+----------+------------+-----------+-------------+--------+-------------+---------+--------+-----------+------+---------------+---------+------------+--------------------+------+--------+--------+-------+
|     1|CA-2016-152156| 11/8/2016|11/11/2016|Second Class|   CG-12520|  Claire Gute|Consumer|United States|Henderson|Kentucky|      42420| South|FUR-BO-10001798|Furniture|   Bookcases|Bush Somerset Col...|261.96|       2|     0.0|41.9136|
|     2|CA-2016-152156| 11/8/2016|11/11/2016

In [ ]:
print('Filtering with PySpark method')
df.filter(df["Sales"] < 200).show(5)

Filtering with PySpark method
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+------+--------+--------+-------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name| sales|quantity|discount| Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+------+--------+--------+-------+
|     3|CA-2016-138688| 6/12/2016| 6/16/2016|  Second Class|   DV-13045|Darrin Van Huff|Corporate|United States|    Los Angeles|California|      90036|  West|OFF-LA-10000240|Office Supplies|      

In [ ]:
##Filtering with SQL:
spark.sql("SELECT * " \
"           FROM superstore " \
"           WHERE Sales < 200").show(5)

+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+------+--------+--------+-------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|  Customer Name|  Segment|      Country|           City|     State|Postal Code|Region|     Product ID|       Category|Sub-Category|        Product Name| sales|quantity|discount| Profit|
+------+--------------+----------+----------+--------------+-----------+---------------+---------+-------------+---------------+----------+-----------+------+---------------+---------------+------------+--------------------+------+--------+--------+-------+
|     3|CA-2016-138688| 6/12/2016| 6/16/2016|  Second Class|   DV-13045|Darrin Van Huff|Corporate|United States|    Los Angeles|California|      90036|  West|OFF-LA-10000240|Office Supplies|      Labels|Self-Adhesive Add...| 1

In [ ]:
spark.sql("SELECT `Order ID`, Country, City" \
"           FROM superstore " \
"           WHERE Sales < 200").show(5)

+--------------+-------------+---------------+
|      Order ID|      Country|           City|
+--------------+-------------+---------------+
|CA-2016-138688|United States|    Los Angeles|
|US-2015-108966|United States|Fort Lauderdale|
|CA-2014-115812|United States|    Los Angeles|
|CA-2014-115812|United States|    Los Angeles|
|CA-2014-115812|United States|    Los Angeles|
+--------------+-------------+---------------+
only showing top 5 rows



In [ ]:

print('With SQL')
spark.sql("SELECT `Order ID` as ORDERID, Country AS `COU ntry`, City AS CITY" \
"           FROM superstore " \
"           WHERE Sales < 200").show(5)

print('With Spark')
superstore_df.filter(F.col("Sales") < 200) \
    .select(
        F.col("Order ID").alias("ORDERID"),
        F.col("Country").alias("COU ntry"),
        F.col("City")
    ).show(5)

With SQL
+--------------+-------------+---------------+
|       ORDERID|     COU ntry|           CITY|
+--------------+-------------+---------------+
|CA-2016-138688|United States|    Los Angeles|
|US-2015-108966|United States|Fort Lauderdale|
|CA-2014-115812|United States|    Los Angeles|
|CA-2014-115812|United States|    Los Angeles|
|CA-2014-115812|United States|    Los Angeles|
+--------------+-------------+---------------+
only showing top 5 rows

With Spark
+--------------+-------------+---------------+
|       ORDERID|     COU ntry|           City|
+--------------+-------------+---------------+
|CA-2016-138688|United States|    Los Angeles|
|US-2015-108966|United States|Fort Lauderdale|
|CA-2014-115812|United States|    Los Angeles|
|CA-2014-115812|United States|    Los Angeles|
|CA-2014-115812|United States|    Los Angeles|
+--------------+-------------+---------------+
only showing top 5 rows



In [ ]:

print('With SQL')
spark.sql("SELECT `Order ID` as ORDERID, Country AS `COU ntry`, City AS CITY" \
"           FROM superstore " \
"           WHERE Sales < 200 AND Region = 'West'").show(5)

print('With Spark')
superstore_df.filter((F.col("Sales") < 200) & (F.col("Region") == 'West')) \
    .select(
        F.col("Order ID").alias("ORDERID"),
        F.col("Country").alias("COU ntry"),
        F.col("City")
    ).show(5)

With SQL
+--------------+-------------+-----------+
|       ORDERID|     COU ntry|       CITY|
+--------------+-------------+-----------+
|CA-2016-138688|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
+--------------+-------------+-----------+
only showing top 5 rows

With Spark
+--------------+-------------+-----------+
|       ORDERID|     COU ntry|       City|
+--------------+-------------+-----------+
|CA-2016-138688|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
|CA-2014-115812|United States|Los Angeles|
+--------------+-------------+-----------+
only showing top 5 rows



In [ ]:
##GroupBys:

##Multiple columns
from pyspark.sql import functions as F
superstore_df.groupBy("Category", "Sub-Category").agg(
                                                F.count("Category").alias("Num of Orders"),
                                                F.avg("Sales").alias("avg_sales")
                                            ).orderBy(F.col("Category").desc(), F.col("avg_sales").desc()).show()



+---------------+------------+-------------+------------------+
|       Category|Sub-Category|Num of Orders|         avg_sales|
+---------------+------------+-------------+------------------+
|     Technology|     Copiers|           68| 2198.941617647058|
|     Technology|    Machines|          115| 1645.553313043478|
|     Technology|      Phones|          889| 374.2940839954598|
|     Technology| Accessories|          775|215.97460387096785|
|Office Supplies|     Storage|          846| 264.0721218026798|
|Office Supplies|    Supplies|          190| 261.0935795454546|
|Office Supplies|  Appliances|          466|230.75571030042917|
|Office Supplies|     Binders|         1523| 139.5015471039777|
|Office Supplies|   Envelopes|          254| 62.35565040650403|
|Office Supplies|       Paper|         1370| 59.47602052091554|
|Office Supplies|      Labels|          364|34.303054945054946|
|Office Supplies|         Art|          796| 34.06883417085422|
|Office Supplies|   Fasteners|          

In [ ]:

print('With SQL')
spark.sql("SELECT Category, `Sub-Category`, count(Category) AS `Num of Orders`, avg(Sales) as avg_sales" \
"           FROM superstore " \
"           GROUP BY Category, `Sub-Category`"\
"           ORDER BY Category desc, avg(Sales) desc"
            ).show()

With SQL
+---------------+------------+-------------+------------------+
|       Category|Sub-Category|Num of Orders|         avg_sales|
+---------------+------------+-------------+------------------+
|     Technology|     Copiers|           68| 2198.941617647058|
|     Technology|    Machines|          115| 1645.553313043478|
|     Technology|      Phones|          889| 374.2940839954598|
|     Technology| Accessories|          775|215.97460387096785|
|Office Supplies|     Storage|          846| 264.0721218026798|
|Office Supplies|    Supplies|          190| 261.0935795454546|
|Office Supplies|  Appliances|          466|230.75571030042917|
|Office Supplies|     Binders|         1523| 139.5015471039777|
|Office Supplies|   Envelopes|          254| 62.35565040650403|
|Office Supplies|       Paper|         1370| 59.47602052091554|
|Office Supplies|      Labels|          364|34.303054945054946|
|Office Supplies|         Art|          796| 34.06883417085422|
|Office Supplies|   Fasteners| 

In [ ]:
##Joins with SQL
# Create sample datasets
customers_data = [
    (1, "John Smith", "john@email.com", "New York"),
    (2, "Jane Doe", "jane@email.com", "Los Angeles"),
    (3, "Bob Johnson", "bob@email.com", "Chicago"),
    (4, "Alice Williams", "alice@email.com", "Houston")
]
customers_df = spark.createDataFrame(customers_data, ["customer_id", "name", "email", "city"])

orders_data = [
    (101, 1, "2023-01-15", 150.00, "Delivered"),
    (102, 2, "2023-01-16", 200.00, "Delivered"),
    (103, 1, "2023-01-17", 75.00, "Shipped"),
    (104, 3, "2023-01-18", 300.00, "Processing"),
    (105, 5, "2023-01-19", 125.00, "Delivered")
]
orders_df = spark.createDataFrame(orders_data, ["order_id", "customer_id", "order_date", "amount", "status"])

# Register the CORRECT DataFrames
customers_df.createOrReplaceTempView("customer_data")  
orders_df.createOrReplaceTempView("orders_data")       


In [ ]:
spark.sql(
    "SELECT * " \
    "FROM customer_data as cd, orders_data as od " \
    "WHERE cd.customer_id = od.customer_id"
).show()

+-----------+-----------+--------------+-----------+--------+-----------+----------+------+----------+
|customer_id|       name|         email|       city|order_id|customer_id|order_date|amount|    status|
+-----------+-----------+--------------+-----------+--------+-----------+----------+------+----------+
|          1| John Smith|john@email.com|   New York|     101|          1|2023-01-15| 150.0| Delivered|
|          1| John Smith|john@email.com|   New York|     103|          1|2023-01-17|  75.0|   Shipped|
|          2|   Jane Doe|jane@email.com|Los Angeles|     102|          2|2023-01-16| 200.0| Delivered|
|          3|Bob Johnson| bob@email.com|    Chicago|     104|          3|2023-01-18| 300.0|Processing|
+-----------+-----------+--------------+-----------+--------+-----------+----------+------+----------+



In [ ]:
spark.sql(
    "SELECT * " \
    "FROM customer_data as cd " \
    "JOIN orders_data as od ON cd.customer_id = od.customer_id"
).show()

+-----------+-----------+--------------+-----------+--------+-----------+----------+------+----------+
|customer_id|       name|         email|       city|order_id|customer_id|order_date|amount|    status|
+-----------+-----------+--------------+-----------+--------+-----------+----------+------+----------+
|          1| John Smith|john@email.com|   New York|     101|          1|2023-01-15| 150.0| Delivered|
|          1| John Smith|john@email.com|   New York|     103|          1|2023-01-17|  75.0|   Shipped|
|          2|   Jane Doe|jane@email.com|Los Angeles|     102|          2|2023-01-16| 200.0| Delivered|
|          3|Bob Johnson| bob@email.com|    Chicago|     104|          3|2023-01-18| 300.0|Processing|
+-----------+-----------+--------------+-----------+--------+-----------+----------+------+----------+



In [ ]:
spark.sql(
    "SELECT * " \
    "FROM customer_data as cd " \
    "LEFT JOIN orders_data as od ON cd.customer_id = od.customer_id"
).show()

+-----------+--------------+---------------+-----------+--------+-----------+----------+------+----------+
|customer_id|          name|          email|       city|order_id|customer_id|order_date|amount|    status|
+-----------+--------------+---------------+-----------+--------+-----------+----------+------+----------+
|          1|    John Smith| john@email.com|   New York|     103|          1|2023-01-17|  75.0|   Shipped|
|          1|    John Smith| john@email.com|   New York|     101|          1|2023-01-15| 150.0| Delivered|
|          2|      Jane Doe| jane@email.com|Los Angeles|     102|          2|2023-01-16| 200.0| Delivered|
|          3|   Bob Johnson|  bob@email.com|    Chicago|     104|          3|2023-01-18| 300.0|Processing|
|          4|Alice Williams|alice@email.com|    Houston|    NULL|       NULL|      NULL|  NULL|      NULL|
+-----------+--------------+---------------+-----------+--------+-----------+----------+------+----------+



In [ ]:
# Union 
sales_q1 = spark.createDataFrame([
    (1, "2023-01-15", 1000),
    (2, "2023-02-20", 1500),
    (3, "2023-03-20", 2500)
], ["sale_id", "date", "amount"])

sales_q2 = spark.createDataFrame([
    (3, "2023-03-20", 2500),
    (4, "2023-04-10", 2600),
    (5, "2023-05-15", 1800),
    (6, "2023-06-20", 9500)
], ["sale_id", "date", "amount"])

# Union (includes duplicates)

sales_q1.createOrReplaceTempView("sales_q1")  
sales_q2.createOrReplaceTempView("sales_q2")       

# UNION - removes duplicates
spark.sql(
    "SELECT * FROM sales_q1 " \
    "UNION " \
    "SELECT * FROM sales_q2"
).show()


# UNION ALL - keeps duplicates (faster)
spark.sql(
    "SELECT * FROM sales_q1 " \
    "UNION ALL " \
    "SELECT * FROM sales_q2"
).show()

+-------+----------+------+
|sale_id|      date|amount|
+-------+----------+------+
|      1|2023-01-15|  1000|
|      2|2023-02-20|  1500|
|      3|2023-03-20|  2500|
|      4|2023-04-10|  2600|
|      5|2023-05-15|  1800|
|      6|2023-06-20|  9500|
+-------+----------+------+

+-------+----------+------+
|sale_id|      date|amount|
+-------+----------+------+
|      1|2023-01-15|  1000|
|      2|2023-02-20|  1500|
|      3|2023-03-20|  2500|
|      3|2023-03-20|  2500|
|      4|2023-04-10|  2600|
|      5|2023-05-15|  1800|
|      6|2023-06-20|  9500|
+-------+----------+------+



In [ ]:
##To see what views are on the environment
spark.catalog.listTables()

[Table(name='customer_data', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='orders_data', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='sales_q1', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='sales_q2', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='superstore', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

## To Python objects:

In [ ]:
##To Python objects
##Pandas
customers_df.toPandas()



,customer_id,name,email,city
0,1,John Smith,john@email.com,New York
1,2,Jane Doe,jane@email.com,Los Angeles
2,3,Bob Johnson,bob@email.com,Chicago
3,4,Alice Williams,alice@email.com,Houston


In [ ]:
##To lists
rows_list = customers_df.collect()
rows_list

[Row(customer_id=1, name='John Smith', email='john@email.com', city='New York'),
 Row(customer_id=2, name='Jane Doe', email='jane@email.com', city='Los Angeles'),
 Row(customer_id=3, name='Bob Johnson', email='bob@email.com', city='Chicago'),
 Row(customer_id=4, name='Alice Williams', email='alice@email.com', city='Houston')]

In [ ]:
for row in rows_list:
    print(row['name'], row['email'])

John Smith john@email.com
Jane Doe jane@email.com
Bob Johnson bob@email.com
Alice Williams alice@email.com


In [ ]:
##List of dictionaries:
list_of_dicts = [row.asDict() for row in customers_df.collect()]
list_of_dicts

[{'customer_id': 1,
  'name': 'John Smith',
  'email': 'john@email.com',
  'city': 'New York'},
 {'customer_id': 2,
  'name': 'Jane Doe',
  'email': 'jane@email.com',
  'city': 'Los Angeles'},
 {'customer_id': 3,
  'name': 'Bob Johnson',
  'email': 'bob@email.com',
  'city': 'Chicago'},
 {'customer_id': 4,
  'name': 'Alice Williams',
  'email': 'alice@email.com',
  'city': 'Houston'}]

**Important Warnings**

Memory: collect() and toPandas() bring all data to the driver node. For large datasets, this can cause out-of-memory errors.

Use Case: Only extract to Python objects when:

- Dataset is small enough to fit in memory
- You need final results for visualization or reporting
- You're doing analysis that requires Pandas/Python libraries



---
<a name="anchorMissing" style="position:absolute;"></a>
## 3.2 Handling Missing Data
---

Here we revisit the data we used in the EDA and Visualisation Workbook to compare how we handle the same data in PySpark.

In [259]:
#df = spark.read.csv("../DataSources/Production.Product.csv",sep='\t',header=True,inferSchema=True)

In [ ]:
import pandas as pd
import numpy as np
import csv
from pyspark.sql import functions as F, types as T
# Assuming 'spark' is already defined (your SparkSession)
url = "https://rockborne-bucket-01-cbs.s3.eu-west-2.amazonaws.com/DataSources/Production.Product.csv"
dfp = pd.read_csv(
    url,
    sep="\t",
    encoding="utf-8-sig",
    engine="python",
    quoting=csv.QUOTE_NONE,
    na_values=["NaN","NAN","NULL","Null","None","", " "],  # treat as missing
    keep_default_na=True
)

# remove wrapping quotes from strings
#dfp = dfp.apply(lambda s: s.str.strip('"') if s.dtype == "object" else s)
##Creating the dataFrame
df = spark.createDataFrame(dfp)

MISSING = ["NaN", "NAN", "NULL", "Null", "None", "", " "]  # tokens to null out

# Identify column types
string_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, T.StringType)]
numeric_cols = [f.name for f in df.schema.fields if isinstance(f.dataType, (T.IntegerType, T.LongType, T.FloatType, T.DoubleType, T.DecimalType))]

# Apply transformations based on column type
selection_list = []
for c in df.columns:
    if c in string_cols:
        # Transformation for String Columns: Trim, then null out MISSING tokens
        selection_list.append(
            F.when(F.trim(F.col(c)).isin(MISSING), None)
             .otherwise(F.col(c))
             .alias(c)
        )
    elif c in numeric_cols:
        # Transformation for Numeric Columns: Explicitly set NaN/inf values to None (null)
        # Note: Spark often handles numeric nulls, but this is explicit.
        # We can also handle specific string tokens if the numeric columns are still messy.
        selection_list.append(
            F.when(F.col(c).isNull() | F.isnan(F.col(c)) | F.col(c).isin([float('inf'), float('-inf')]), None)
             .otherwise(F.col(c))
             .alias(c)
        )
    else:
        # No change for other types (e.g., Date, Boolean)
        selection_list.append(F.col(c))

# Apply the transformations
df = df.select(selection_list)

### **Finding missing data**

In [262]:
df.show()

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

In [263]:
df.select(df['Color'].isNull()).show()

+---------------+
|(Color IS NULL)|
+---------------+
|           true|
|           true|
|           true|
|           true|
|           true|
|          false|
|          false|
|          false|
|          false|
|          false|
|          false|
|           true|
|           true|
|           true|
|           true|
|           true|
|           true|
|           true|
|           true|
|           true|
+---------------+
only showing top 20 rows



How do we find out the rows with the missing values?

In [264]:
df.where(df['Color'].isNull()).show()

+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+----

We can use list comprehension in combination with our spark commands to find all of the nulls in the dataframe on a column by column basis

Reference: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/functions.html

In [265]:
# Find Count of Null, None, NaN of All DataFrame Columns
df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
|ProductID|Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|SellStartDate|SellEndDate|DiscontinuedDate|rowguid|ModifiedDate|
+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
|        0|   0|            0|       0|                0|  24

### **Drop missing data**

In [266]:
df.show()

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

Dropping any rows that contain missing data:

In [267]:
df.na.drop().show()

+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
|ProductID|Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|SellStartDate|SellEndDate|DiscontinuedDate|rowguid|ModifiedDate|
+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
+---------+----+-------------+--------+-----------------+----

Dropping rows that have at least 22 **NON**-null values:

In [268]:
#type(df.columns)
#len(df.columns)
df.na.drop(thresh=22).show()

+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-------------------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|        SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+--------

Dropping any rows where the Color column value is null:

In [269]:
df.na.drop(subset=["Color"]).show()


+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-------------------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|        SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-----

Dropping any non-complete rows:

In [270]:
df.na.drop(how='any').show()

+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
|ProductID|Name|ProductNumber|MakeFlag|FinishedGoodsFlag|Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|SellStartDate|SellEndDate|DiscontinuedDate|rowguid|ModifiedDate|
+---------+----+-------------+--------+-----------------+-----+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------+-----------+----------------+-------+------------+
+---------+----+-------------+--------+-----------------+----

Dropping columns:

In [271]:
df.drop('DiscontinuedDate').show()

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+--------------------+--

### **Fill in missing values**

All string type columns:

In [272]:
df.na.fill('NEW VALUE').show()

+---------+--------------------+-------------+--------+-----------------+---------+----------------+------------+------------+---------+---------+-------------------+---------------------+------+-----------------+-----------+---------+---------+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag|    Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|     Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|    Class|    Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+---------+----------------+------------+------------+---------+---------+-------------------+---------------------+------+-----------------+-----------+---------+---------+---------

All numeric type columns:

In [273]:
df.na.fill(0).show()

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

Specified columns filled with the subset parameter:

In [274]:
df.na.fill('No Color',subset=['Color']).show()

+---------+--------------------+-------------+--------+-----------------+--------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag|   Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+--------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+---------------

Filling with mean values:

In [275]:
#from pyspark.sql.functions import mean
mean_val = df.select(F.mean(df['Weight'])).collect()

In [276]:
mean_val

[Row(avg(Weight)=74.06921951219513)]

In [277]:
# Display mean value
mean_weight = mean_val[0][0]
mean_weight

74.06921951219513

In [278]:
df.na.fill(mean_weight,["Weight"]).show()

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+-----------------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|           Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+-----------------+-----------------+-----------+-----+-----+--------------------+---

**Note:** None of these changes will save unless you reassign

e.g. `df = df.na.fill(mean_weight, ['Weight']).show()`

In [279]:
#***Task***
# Fill missing values in the WeightUnitMeasureCode column with 'CM'

#-----------------------------------------------------------------#

df.na.fill('CM', subset=['WeightUnitMeasureCode']).show()
#-----------------------------------------------------------------#

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+----+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-

<details>
    <summary style="color:green;font-weight:bold">Click here for the answer to the task</summary>
    
>
```
df.na.fill('CM', subset=['WeightUnitMeasureCode']).show()
```
</details>

In [280]:
#***Task***
# Fill missing values in the Size column with 'NoSize'

#-----------------------------------------------------------------#

df.na.fill('NoSize', subset=['Size']).show()
#-----------------------------------------------------------------#

+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+------+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+-------------------+-----------+----------------+--------------------+--------------------+
|ProductID|                Name|ProductNumber|MakeFlag|FinishedGoodsFlag| Color|SafetyStockLevel|ReorderPoint|StandardCost|ListPrice|  Size|SizeUnitMeasureCode|WeightUnitMeasureCode|Weight|DaysToManufacture|ProductLine|Class|Style|ProductSubcategoryID|ProductModelID|      SellStartDate|SellEndDate|DiscontinuedDate|             rowguid|        ModifiedDate|
+---------+--------------------+-------------+--------+-----------------+------+----------------+------------+------------+---------+------+-------------------+---------------------+------+-----------------+-----------+-----+-----+--------------------+--------------+---------------

<details>
    <summary style="color:green;font-weight:bold">Click here for the answer to the task</summary>
    
>
```
df.na.fill('NoSize', subset=['Size']).show()
```
</details>

## **Writing Data**

#### Writing a PySpark DataFrame to CSV: 3 Methods

##### 1. Using `.write.csv()` (Default PySpark Method)  
- **Output**: A **folder** containing multiple CSV files and a `_SUCCESS` file.  
- **When to Use**: Best for **large datasets** and distributed environments where partitioning is beneficial.  

---

##### 2. Using `.write.csv()` with `.coalesce(1)` (Single File Output)  
- **Output**: A **folder** containing **one** CSV file.  
- **When to Use**: When a **single CSV file** is needed, but still working in a distributed environment.  

---

##### 3. Using `.toPandas().to_csv()` (Pandas Method)  
- **Output**: A **single CSV file**.  
- **When to Use**: Suitable for **small datasets** that fit into memory; easier for handling as a single file.  

---

#### Choosing the Right Method  

| Method                        | Output Type        | Best for Large Data? | Produces a Single File? |
|--------------------------------|--------------------|----------------------|-------------------------|
| `.write.csv()`                 | Folder w/ multiple files | ✅ Yes | ❌ No  |
| `.coalesce(1).write.csv()`      | Folder w/ one file | ⚠️ Limited | ❌ No  |
| `.toPandas().to_csv()`          | Single CSV file | ❌ No (memory risk) | ✅ Yes |


In [ ]:
##Local files
#df = spark.read.csv('../DataSources/superstore.csv',inferSchema=True,header=True)
# Default:
#df.orderBy(df["Sales"].desc()).write.csv("DescendingSales1.csv")
# Coalesce
#df.orderBy(df["Sales"].desc()).coalesce(1).write.csv("DescendingSales2.csv", header=True)
# Convert to Pandas and save
#df.orderBy(df["Sales"].desc()).toPandas().to_csv("DescendingSales2.csv", index=False)

In [190]:
##AWS S3 Bucket:
import pandas as pd
url = "https://rockborne-bucket-01-cbs.s3.eu-west-2.amazonaws.com/DataSources/superstore.csv"
# Read with pandas
pandas_df = pd.read_csv(url, encoding='latin-1')
# Convert to PySpark
superstore_df = spark.createDataFrame(pandas_df)
#superstore_df.printSchema()
superstore_df = superstore_df.withColumn("sales", F.col("sales").cast("double")) \
       .withColumn("quantity", F.col("quantity").cast("int")) \
       .withColumn("discount", F.col("discount").cast("double"))
superstore_df.printSchema()
superstore_df.show()
superstore_df.orderBy(superstore_df["Sales"].desc()).toPandas().to_csv("DescendingSales2.csv", index=False)

root
 |-- Row ID: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullable = true)
 |-- Ship Mode: string (nullable = true)
 |-- Customer ID: string (nullable = true)
 |-- Customer Name: string (nullable = true)
 |-- Segment: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postal Code: long (nullable = true)
 |-- Region: string (nullable = true)
 |-- Product ID: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Sub-Category: string (nullable = true)
 |-- Product Name: string (nullable = true)
 |-- Sales: double (nullable = true)
 |-- Quantity: long (nullable = true)
 |-- Discount: double (nullable = true)
 |-- Profit: double (nullable = true)

root
 |-- Row ID: long (nullable = true)
 |-- Order ID: string (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Ship Date: string (nullab

<a name="anchorConclusion" style="position:absolute;"></a>
<hr style="border:0.5px solid", color= "gray">

# 4. Conclusion
<hr style="border-top:1px dashed">

- PySpark is a powerful language for Data Science Analysts to learn, because it enables scalabe data wrangling and analysis beyond the limitations of pandas.
- If you already know Python, Pandas and SQL this should be a great jump into big data.
- If you are interested in Data Engineering, [Azure Databricks](https://docs.microsoft.com/en-us/azure/databricks/languages/python) uses PySpark as one of the primary languages to interact with data.
- For more information read the [PySpark Documentation](https://spark.apache.org/docs/latest/api/python/getting_started/index.html).

## Citations
<a name="KDNuggets" style="position:absolute;"></a>
7 Steps to Mastering Apache 2.0 :cite:p:[2016:KDNuggets](https://www.kdnuggets.com/2016/09/7-steps-mastering-apache-spark.html)
<br>